In [50]:
import pandas as pd
direccion = "./../data/raw/prueba.csv"
df = pd.read_csv(direccion, nrows = 10000)
df.head()

df.Cancelled

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
9995    0.0
9996    0.0
9997    0.0
9998    0.0
9999    0.0
Name: Cancelled, Length: 10000, dtype: float64

In [15]:
direccion = "./../data/raw/prueba.csv"

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df = spark.read.csv(direccion, header="true", inferSchema="true").limit(10000)

In [16]:
def clean(df):
    #Pasar a minusculas los nombres de columnas
    for col in df.columns:
        df = df.withColumnRenamed(col, col.lower())

    #Seleccionar columnas no vacias 

    base = df.select(df.year,df.quarter, df.month, df.dayofmonth, df.dayofweek, df.flightdate, df.reporting_airline, df.dot_id_reporting_airline, df.iata_code_reporting_airline, df.tail_number, df.flight_number_reporting_airline, df.originairportid, df.originairportseqid, df.origincitymarketid, df.origin, df.origincityname, df.originstate, df.originstatefips, df.originstatename, df.originwac, df.destairportid, df.destairportseqid, df.destcitymarketid, df.dest, df.destcityname, df.deststate, df.deststatefips, df.deststatename, df.destwac, df.crsdeptime, df.deptime, df.depdelay, df.depdelayminutes, df.depdel15, df.departuredelaygroups, df.deptimeblk, df.taxiout, df.wheelsoff, df.wheelson, df.taxiin, df.crsarrtime, df.arrtime, df.arrdelay, df.arrdelayminutes, df.arrdel15, df.arrivaldelaygroups, df.arrtimeblk, df.cancelled, df.diverted, df.crselapsedtime, df.actualelapsedtime, df.airtime, df.flights, df.distance, df.distancegroup, df.divairportlandings )

    #agregar columna con clasificación de tiempo en horas de atraso del vuelo 0-1.5, 1.5-3.5,3.5-, cancelled

    from pyspark.sql import functions as f
    base = base.withColumn('rangoatrasohoras', f.when(f.col('cancelled') == 1, "cancelled").when(f.col('depdelayminutes') < 90, "0-1.5").when((f.col('depdelayminutes') > 90) & (f.col('depdelayminutes')<210), "1.5-3.5").otherwise("3.5-"))

    from pyspark.sql.functions import udf
    from pyspark.sql.types import StringType
    from pyspark.sql.functions import col, lower, regexp_replace, split

    #Función limpieza
    def clean_text(c):
        c = lower(c)
        c = regexp_replace(c, " ", "_")
        c = f.split(c, '\,')[0]
        return c


     # Aplicación de la función limpieza
    base = base.withColumn("origincityname", clean_text(col("origincityname")))
    base = base.withColumn("destcityname", clean_text(col("destcityname")))
    return base

In [17]:
df_clean = clean(df)

In [86]:
type(df_clean)
from pyspark.sql.types import IntegerType, DoubleType

double_cols = [f.name for f in df_clean.schema.fields if isinstance(f.dataType, DoubleType)]
int_cols = [f.name for f in df_clean.schema.fields if isinstance(f.dataType, IntegerType)]
num_cols = int_cols + double_cols

cols = num_cols + ['rangoatrasohoras']

df_num = df_clean[cols]

# MISSING VALUES
# POR AHORA ZEROS (MANEJAR ESTO DESPUES)
#df_na = df_num.fill(0)

numericals_imputed = [var + "_imputed" for var in double_cols]

from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols = double_cols, outputCols = numericals_imputed)
df_impute = imputer.setStrategy("median").fit(df_num).transform(df_num)
new_vars = numericals_imputed + int_cols + ['rangoatrasohoras']
df_impute = df_impute[new_vars]

df_impute = df_impute.fillna(0, subset=int_cols)

df_impute.columns


['depdelay_imputed',
 'depdelayminutes_imputed',
 'depdel15_imputed',
 'taxiout_imputed',
 'taxiin_imputed',
 'arrdelay_imputed',
 'arrdelayminutes_imputed',
 'arrdel15_imputed',
 'cancelled_imputed',
 'diverted_imputed',
 'crselapsedtime_imputed',
 'actualelapsedtime_imputed',
 'airtime_imputed',
 'flights_imputed',
 'distance_imputed',
 'year',
 'quarter',
 'month',
 'dayofmonth',
 'dayofweek',
 'dot_id_reporting_airline',
 'flight_number_reporting_airline',
 'originairportid',
 'originairportseqid',
 'origincitymarketid',
 'originstatefips',
 'originwac',
 'destairportid',
 'destairportseqid',
 'destcitymarketid',
 'deststatefips',
 'destwac',
 'crsdeptime',
 'deptime',
 'departuredelaygroups',
 'wheelsoff',
 'wheelson',
 'crsarrtime',
 'arrtime',
 'arrivaldelaygroups',
 'distancegroup',
 'divairportlandings',
 'rangoatrasohoras']

In [88]:
from pyspark.sql.functions import monotonically_increasing_id

# This will return a new DF with all the columns + id
df_id  = df_impute.withColumn("id",monotonically_increasing_id())

nuevas_vars =  [x for x in df_id.columns if x not in ['rangoatrasohoras']]

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=nuevas_vars ,outputCol="features")
assembler = assembler.transform(df_id)

keep_vars = ["id", "features", 'rangoatrasohoras']
df_assembler = assembler[keep_vars]
df_assembler.show(2)

+---+--------------------+----------------+
| id|            features|rangoatrasohoras|
+---+--------------------+----------------+
|  0|[-4.0,0.0,0.0,9.0...|           0-1.5|
|  1|[18.0,18.0,1.0,13...|           0-1.5|
+---+--------------------+----------------+
only showing top 2 rows



## Modelado

###  Formato al Dataframe 

In [ ]:
import org.apache.spark.ml.feature.OneHotEncoderEstimator

In [100]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer


# Esta funcion agrega un id numerico a cada valor diferente de un valor categorico 
# es como establecer los niveles en R de una factor pero los niveles son numericos,
# sus id. El indice se establece por orden de frecuencia (descendente), por lo que 
# el indice 0 corresponde a la variable que aparece con mas frecuencia
string_indexer = StringIndexer(inputCol='rangoatrasohoras', outputCol="categoryIndex")
model = string_indexer.fit(df_assembler)
indexed = model.transform(df_assembler)

# Antes falta escalar variables
# OneHotEncoder no tiene un fit ya que solo es un transformador
encoder = OneHotEncoder(inputCol="categoryIndex", outputCol="categoryVec")
encoded = encoder.transform(indexed )
df_encoded = encoded[["id", "features", "categoryIndex","categoryVec"]]

df_encoded.show(10)

+---+--------------------+-------------+-------------+
| id|            features|categoryIndex|  categoryVec|
+---+--------------------+-------------+-------------+
|  0|[-4.0,0.0,0.0,9.0...|          0.0|(3,[0],[1.0])|
|  1|[18.0,18.0,1.0,13...|          0.0|(3,[0],[1.0])|
|  2|[36.0,36.0,1.0,12...|          0.0|(3,[0],[1.0])|
|  3|[36.0,36.0,1.0,15...|          0.0|(3,[0],[1.0])|
|  4|[-7.0,0.0,0.0,8.0...|          0.0|(3,[0],[1.0])|
|  5|[-4.0,0.0,0.0,22....|          0.0|(3,[0],[1.0])|
|  6|[1.0,1.0,0.0,11.0...|          0.0|(3,[0],[1.0])|
|  7|[12.0,12.0,0.0,17...|          0.0|(3,[0],[1.0])|
|  8|[166.0,166.0,1.0,...|          1.0|(3,[1],[1.0])|
|  9|[-2.0,0.0,0.0,22....|          0.0|(3,[0],[1.0])|
+---+--------------------+-------------+-------------+
only showing top 10 rows



In [119]:
# Podemos empezar con el modelado
#from pyspark.sql import randomSplit

# Separamos en train y test
df_train, df_test = df_encoded.randomSplit([0.8,0.2], 123)
df_test.show(2)

+---+--------------------+-------------+-------------+
| id|            features|categoryIndex|  categoryVec|
+---+--------------------+-------------+-------------+
|  1|[18.0,18.0,1.0,13...|          0.0|(3,[0],[1.0])|
|  2|[36.0,36.0,1.0,12...|          0.0|(3,[0],[1.0])|
+---+--------------------+-------------+-------------+
only showing top 2 rows



In [120]:

from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit


selector = ChiSqSelector(numTopFeatures=10, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="categoryIndex")

selector_model = selector.fit(df_train)

selector_result = selector_model.transform(df_train)
print("ChiSqSelector output with top %d features selected" % selector.getNumTopFeatures())

df_train = selector_result[["id", "selectedFeatures", "categoryIndex","categoryVec"]]

df_train.show(2)

selector_result2 = selector_model.transform(df_test)

df_test = selector_result2[["id", "selectedFeatures", "categoryIndex","categoryVec"]]


ChiSqSelector output with top 10 features selected
+---+--------------------+-------------+-------------+
| id|    selectedFeatures|categoryIndex|  categoryVec|
+---+--------------------+-------------+-------------+
|  0|[-4.0,0.0,0.0,9.0...|          0.0|(3,[0],[1.0])|
|  4|[-7.0,0.0,0.0,8.0...|          0.0|(3,[0],[1.0])|
+---+--------------------+-------------+-------------+
only showing top 2 rows



In [121]:
# Configuramos el estimator StandarScaler como lo necesitamos (por default
# withMean esta en False porque hace que se regrese un vector dense...
# hay que tener cuidado con eso cuando estemos manejandoo vectores sparse
scaler = StandardScaler(inputCol="selectedFeatures", outputCol="scaled_features",
                        withStd=True, withMean=True)
# Creamos el modelo StandardScaler para los datos de entrada
scaler_model = scaler.fit(df_train)

# Transformamos los datos 
scaled_data = scaler_model.transform(df_train)

df_train = scaled_data[["id", "scaled_features", "categoryIndex","categoryVec"]]
df_train.show(2)

scaled_data_test = scaler_model.transform(df_test)
df_test = scaled_data_test[["id", "scaled_features", "categoryIndex","categoryVec"]]


+---+--------------------+-------------+-------------+
| id|     scaled_features|categoryIndex|  categoryVec|
+---+--------------------+-------------+-------------+
|  0|[-0.1967629108606...|          0.0|(3,[0],[1.0])|
|  4|[-0.2663808227574...|          0.0|(3,[0],[1.0])|
+---+--------------------+-------------+-------------+
only showing top 2 rows



In [130]:
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# instantiate the base classifier.
lr = LogisticRegression(maxIter=10, regParam=0.01, fitIntercept=True , 
                        labelCol ="categoryIndex", featuresCol = "scaled_features"  )

# instantiate the One Vs Rest Classifier.
ovr = OneVsRest(classifier=lr,  labelCol ="categoryIndex", featuresCol = "scaled_features" )

# train the multiclass model.
ovrModel = ovr.fit(df_train)

# score the model on test data.
predictions = ovrModel.transform(df_test)

# obtain evaluator.
evaluator = MulticlassClassificationEvaluator(metricName="accuracy",labelCol ="categoryIndex")

# compute the classification error on test data.
accuracy = evaluator.evaluate(predictions )
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.0126459


## Util para obtener documentación y ver parámetros

In [131]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LogisticRegression, OneVsRest

lr = LogisticRegression(maxIter=10, regParam=0.01,
                       labelCol ="categoryVec", featuresCol = "scaled_features"  )

# instantiate the One Vs Rest Classifier.
ovr = OneVsRest(classifier=lr)

print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")
print("LogisticRegression parameters:\n" +evaluator.explainParams() + "\n")


LogisticRegression parameters:
aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: scaled_features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: categoryVec)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitti